In [1]:
from dotenv import load_dotenv
load_dotenv('.env')

True

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model='llama3.2:latest', base_url='http://localhost:11434')

### Custom Tools

In [28]:
from langchain_core.tools import tool

@tool
def add(a, b):
    """
    Suma dos enteros

    Args:
    a: Primer entero
    b: Segundo entero
    """
    return a + b

@tool
def multiply(a, b):
    """
    Multiplica dos enteros

    Args:
    a: Primer entero
    b: Segundo entero
    """
    return a * b


In [4]:
add.name, add.description, add.args, add.model_json_schema

('add',
 'Suma dos enteros\n\nArgs:\na: Primer entero\nb: Segundo entero',
 {'a': {'title': 'A'}, 'b': {'title': 'B'}},
 <bound method BaseModel.model_json_schema of <class 'langchain_core.tools.structured.StructuredTool'>>)

In [5]:
multiply.invoke({'a': 32, 'b': 37})

1184

In [6]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools=tools)

In [8]:
question = '¿Cuál es resultado de multiplicar de 3 por 5?'
llm_with_tools.invoke(question).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'ea1837de-c74d-4c8e-8a8e-0a49fee54464',
  'type': 'tool_call'}]

### Calling in-built toll
##### DuckDuckGo Search

In [11]:
# pip install duckduckgo-search wikipedia xmltodict tavily-python

from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.invoke("¿Cuál es el nombre del actual presidente de Chile?")

'El exlíder estudiantil, de 35 años, representa la parte de la sociedad chilena que quiere un "cambio profundo" y que encabezó las protestas por la igualdad de derechos en 2019. El exlíder estudiantil Gabriel Boric fue elegido este domingo (19 de diciembre) como el hombre más joven en la historia de Chile. Su victoria representa un giro ... De acuerdo a información de Trasparencia, actualmente el Presidente de la República de Chile -Gabriel Boric- tiene una renta bruta de $5.742.240. Ese monto, sin embargo, no es el más alto ... ¿Cuánto le queda a Gabriel Boric en el Gobierno? El abogado y actual presidente Gabriel Boric le queda en la presidencia actualmente un año y tres meses, ya que el mandato presidencial en Chile tiene una duración de cuatro años por lo que su término está previsto para el 11 de marzo de 2026.. En sus primeros años, Boric ha enfrentado dificultades como la baja aprobación popular y la ... S.E. el Presidente de la República, Gabriel Boric Font, se reúne con la min

#### Tavily Search

In [12]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
    max_results = 3,
    search_depth='advanced',
    include_answer=True,
    include_raw_content=True,
)

question = "¿Cuál es el nombre del presidente de Chile?"
search.invoke(question)

[{'url': 'https://www.bcn.cl/historiapolitica/presidentes_de_la_republica',
  'content': '##### Manuel Blanco EncaladaPresidente de la República entre el 9 de julio de 1826 y el 9 de septiembre de 1826. ##### Jorge Montt ÁlvarezPresidente de la República entre el 26 de diciembre de 1891 y el 18 de septiembre de 1896. ##### Eduardo Frei MontalvaPresidente de la República entre el 3 de noviembre de 1964 y el 3 de noviembre de 1970. ##### Salvador Allende GossensPresidente de la República entre el 3 de noviembre de 1970 y el 11 de septiembre de 1973. ##### Patricio Aylwin AzócarPresidente de la República entre el 11 de marzo de 1990 y el 11 de marzo de 1994. ##### Eduardo Frei Ruiz-TaglePresidente de la República entre el 11 de marzo de 1994 y el 11 de marzo de 2000.'},
 {'url': 'https://x.com/gabrielboric',
  'content': "Presidente de la República de Chile, período 2022-2026.'La duda debe seguir a la convicción como una sombra' A. Camus."},
 {'url': 'https://www.instagram.com/gabrielbori

### Wikipedia Search

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

#query = 'what is the capital of Chile?'
query = 'What is operations research?'
print(wikipedia.invoke(query))

Page: Operations research
Summary: Operations research (British English: operational research) (U.S. Air Force Specialty Code: Operations Analysis), often shortened to the initialism OR, is a branch of applied mathematics that deals with the development and application of analytical methods to improve management and decision-making. Although the term management science is sometimes used similarly, the two fields differ in their scope and emphasis.
Employing techniques from other mathematical sciences, such as modeling, statistics, and optimization, operations research arrives at optimal or near-optimal solutions to decision-making problems. Because of its emphasis on practical applications, operations research has overlapped with many other disciplines, notably industrial engineering. Operations research is often concerned with determining the extreme values of some real-world objective: the maximum (of profit, performance, or yield) or minimum (of loss, risk, or cost). Originating in 

In [15]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

search_pubmed = PubmedQueryRun()
search_pubmed.invoke("What is the latest research on Cancer?")

'No good PubMed Result was found'

### Tool Calling with LLM

In [16]:
@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information

    Args: 
    query: The search query
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query) 
    return response

@tool
def pubmed_search(query):
    """
    Search pubmed for medical and life sciences queries

    Args: 
    query: The search query
    """
    
    search = PubmedQueryRun()
    response = search.invoke(query) 
    return response

@tool
def tavi_search(query):
    """
    Search pubmed for realtime and lates information.
    For example, news on IA

    Args: 
    query: The search query
    """
    
    search = TavilySearchResults(
        max_results=5,
        search_depth="advance",
        include_answer=True,
        include_raw_content=True
    
    )
    response = search.invoke(query) 
    return response

In [17]:
tools = [wikipedia_search, pubmed_search, tavi_search]

list_of_tools = { tool.name: tool for tool in tools }


In [18]:
list_of_tools

{'wikipedia_search': StructuredTool(name='wikipedia_search', description='Search wikipedia for general information\n\nArgs: \nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.wikipedia_search'>, func=<function wikipedia_search at 0x000002B326D9E480>),
 'pubmed_search': StructuredTool(name='pubmed_search', description='Search pubmed for medical and life sciences queries\n\nArgs: \nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.pubmed_search'>, func=<function pubmed_search at 0x000002B320DD0CC0>),
 'tavi_search': StructuredTool(name='tavi_search', description='Search pubmed for realtime and lates information.\nFor example, news on IA\n\nArgs: \nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.tavi_search'>, func=<function tavi_search at 0x000002B3233A72E0>)}

In [19]:
llm_with_tools = llm.bind_tools(tools=tools)

In [20]:
query = 'What is the latest news?'
response = llm_with_tools.invoke(query)

In [21]:
print(response.tool_calls)

[{'name': 'tavi_search', 'args': {'query': 'latest news'}, 'id': '332af74a-9e48-4bc8-bd15-ca1c207d09b3', 'type': 'tool_call'}]


### Final Result with Tool Calling

In [22]:
from langchain_core.messages import HumanMessage

In [31]:
query = '¿What is 13 * 3?'
messages = [HumanMessage(query)]
tool_calls = llm_with_tools.invoke(messages).tool_calls

In [32]:
tool_calls

[{'name': 'wikipedia_search',
  'args': {'query': '13 * 3'},
  'id': '9f4d529f-3751-4ec9-a19b-d1c6af82f696',
  'type': 'tool_call'}]